# Fire Response & Management Capacity Gap Analysis for Tribal Lands

This notebook analyzes fire response capacity and governance gaps on Tribal lands by mapping:
- Tribal fire programs and staffing
- Nearby federal/state fire stations
- Response times based on road networks
- Service gaps and coverage areas

**Outputs:**
- Interactive maps showing response gaps
- Network-based isochrones (reachable areas within X minutes)
- Coverage analysis for Tribal lands

## Imports

In [1]:
# Install required packages (uncomment if needed) or use tribal_fire.yaml to create project environment.
# !pip install geopandas folium osmnx networkx pandas matplotlib contextily requests shapely rtree

In [2]:
# Imports
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import contextily as ctx
import warnings
from datetime import datetime
import json

warnings.filterwarnings('ignore')

print("Libraries loaded successfully")

Libraries loaded successfully


## Data Loading and Cleaning

### Load Tribal Lands Boundaries

In [3]:
# Option 1: Load from file (shapefile, GeoJSON, etc.)
# tribal_lands = gpd.read_file('path/to/tribal_lands.shp')

# Option 2: Download from public sources (example using BIA data)
# tribal_lands_url = "https://biamaps.doi.gov/arcgis/rest/services/..."

# Example: Create sample Tribal lands for demonstration
# Replace this with your actual data
sample_tribal_data = {
    'tribe_name': ['Navajo Nation', 'Fort Apache', 'San Carlos', 'Tohono O\'odham'],
    'state': ['AZ', 'AZ', 'AZ', 'AZ'],
    'has_fire_program': [True, True, False, True],
    'fire_staff_count': [45, 12, 0, 8],
    'lon': [-110.0, -110.3, -110.5, -111.8],
    'lat': [36.0, 33.8, 33.3, 32.0]
}

# Create sample geometries (simplified circles for demonstration)
tribal_lands = gpd.GeoDataFrame(
    sample_tribal_data,
    geometry=[Point(lon, lat).buffer(0.3) for lon, lat in 
              zip(sample_tribal_data['lon'], sample_tribal_data['lat'])],
    crs='EPSG:4326'
)

print(f"Loaded {len(tribal_lands)} Tribal land areas")
print("\nTribal lands summary:")
display(tribal_lands.head())

Loaded 4 Tribal land areas

Tribal lands summary:


,tribe_name,state,has_fire_program,fire_staff_count,lon,lat,geometry
0,Navajo Nation,AZ,True,45,-110.0,36.0,"POLYGON ((-109.70000 36.00000, -109.70144 35.9..."
1,Fort Apache,AZ,True,12,-110.3,33.8,"POLYGON ((-110.00000 33.80000, -110.00144 33.7..."
2,San Carlos,AZ,False,0,-110.5,33.3,"POLYGON ((-110.20000 33.30000, -110.20144 33.2..."
3,Tohono O'odham,AZ,True,8,-111.8,32.0,"POLYGON ((-111.50000 32.00000, -111.50144 31.9..."


### Load Fire Station Data

In [4]:
# Option 1: Load from existing data sources
# federal_stations = gpd.read_file('path/to/federal_fire_stations.shp')
# state_stations = gpd.read_file('path/to/state_fire_stations.shp')

# Option 2: Query OpenStreetMap for fire stations
def get_fire_stations_osm(bbox, station_type='all'):
    """
    Query OpenStreetMap for fire stations within a bounding box.
    
    Parameters:
    bbox: tuple (north, south, east, west) in lat/lon
    station_type: 'all', 'government', or 'volunteer'
    """
    try:
        tags = {'amenity': 'fire_station'}
        stations = ox.features_from_bbox(bbox=bbox, tags=tags)
        
        if isinstance(stations, gpd.GeoDataFrame):
            # Convert to points (use centroid for polygons)
            stations['geometry'] = stations.geometry.centroid
            return stations
        return gpd.GeoDataFrame()
    except Exception as e:
        print(f"Error querying OSM: {e}")
        return gpd.GeoDataFrame()

# Example: Create sample fire stations
# Replace with actual data sources
sample_stations = {
    'station_name': [
        'BIA Fort Defiance Station', 'USFS Alpine Station',
        'AZ State Station 51', 'BLM Globe Station',
        'USFS Tonto Station', 'AZ State Station 23'
    ],
    'agency': ['BIA', 'USFS', 'State', 'BLM', 'USFS', 'State'],
    'lon': [-109.1, -109.2, -110.2, -110.8, -111.2, -111.5],
    'lat': [35.7, 33.9, 33.5, 33.4, 33.8, 32.2],
    'crew_size': [15, 20, 12, 8, 18, 10],
    'equipment': ['Type 1', 'Type 1', 'Type 2', 'Type 3', 'Type 1', 'Type 2']
}

fire_stations = gpd.GeoDataFrame(
    sample_stations,
    geometry=[Point(lon, lat) for lon, lat in 
              zip(sample_stations['lon'], sample_stations['lat'])],
    crs='EPSG:4326'
)

print(f"Loaded {len(fire_stations)} fire stations")
print(f"\nAgency breakdown:")
print(fire_stations['agency'].value_counts())
display(fire_stations.head())

Loaded 6 fire stations

Agency breakdown:
agency
USFS     2
State    2
BIA      1
BLM      1
Name: count, dtype: int64


,station_name,agency,lon,lat,crew_size,equipment,geometry
0,BIA Fort Defiance Station,BIA,-109.1,35.7,15,Type 1,POINT (-109.10000 35.70000)
1,USFS Alpine Station,USFS,-109.2,33.9,20,Type 1,POINT (-109.20000 33.90000)
2,AZ State Station 51,State,-110.2,33.5,12,Type 2,POINT (-110.20000 33.50000)
3,BLM Globe Station,BLM,-110.8,33.4,8,Type 3,POINT (-110.80000 33.40000)
4,USFS Tonto Station,USFS,-111.2,33.8,18,Type 1,POINT (-111.20000 33.80000)


## Road Network Analysis

### Download Road Network

In [ ]:
# Define study area - get bounding box from tribal lands
bounds = tribal_lands.total_bounds  # [minx, miny, maxx, maxy]
bbox = (bounds[3], bounds[1], bounds[2], bounds[0])  # north, south, east, west

print(f"Study area bounding box: {bbox}")
print("Downloading road network from OpenStreetMap...")
print("(This may take several minutes for large areas)")

# Download drivable road network
try:
    # Use drive network for emergency vehicle routing
    G = ox.graph_from_bbox(
        bbox=bbox,
        network_type='drive',
        simplify=True
    )
    
    # Add travel speeds to edges (if not present)
    G = ox.add_edge_speeds(G)
    G = ox.add_edge_travel_times(G)
    
    print(f"✓ Road network downloaded successfully")
    print(f"  Nodes: {len(G.nodes)}")
    print(f"  Edges: {len(G.edges)}")
    
except Exception as e:
    print(f"Error downloading network: {e}")
    print("Creating simplified example network...")
    # Create a simple grid network as fallback
    G = ox.graph_from_bbox(
        bbox=(bounds[3]+0.1, bounds[1]-0.1, bounds[2]+0.1, bounds[0]-0.1),
        network_type='drive',
        simplify=True
    )

Study area bounding box: (36.3, 31.7, -109.7, -112.1)
(This may take several minutes for large areas)


### Calculate Response Time Isochrones

In [ ]:
def calculate_isochrones(G, point, trip_times, speed_multiplier=1.5):
    """
    Calculate network-based isochrones from a point.
    
    Parameters:
    G: NetworkX graph with travel times
    point: shapely Point or (lon, lat) tuple
    trip_times: list of times in minutes (e.g., [10, 20, 30])
    speed_multiplier: factor for emergency vehicle speed (default 1.5x normal)
    
    Returns:
    GeoDataFrame with isochrone polygons
    """
    try:
        # Find nearest network node
        if isinstance(point, Point):
            center_node = ox.distance.nearest_nodes(G, point.x, point.y)
        else:
            center_node = ox.distance.nearest_nodes(G, point[0], point[1])
        
        # Adjust travel times for emergency vehicles
        G_emergency = G.copy()
        for u, v, k, data in G_emergency.edges(keys=True, data=True):
            if 'travel_time' in data:
                data['travel_time'] = data['travel_time'] / speed_multiplier
        
        isochrone_polys = []
        
        for trip_time in sorted(trip_times):
            # Calculate subgraph of nodes reachable within trip_time
            subgraph = nx.ego_graph(
                G_emergency, center_node, 
                radius=trip_time * 60,  # convert to seconds
                distance='travel_time'
            )
            
            # Get node points
            node_points = [Point(G.nodes[node]['x'], G.nodes[node]['y']) 
                          for node in subgraph.nodes()]
            
            # Create convex hull or buffer union
            if len(node_points) > 2:
                # Create buffer around nodes and union
                buffered = [p.buffer(0.01) for p in node_points]
                iso_poly = unary_union(buffered).convex_hull
                isochrone_polys.append({
                    'geometry': iso_poly,
                    'time_minutes': trip_time,
                    'node_count': len(node_points)
                })
        
        return gpd.GeoDataFrame(isochrone_polys, crs='EPSG:4326')
    
    except Exception as e:
        print(f"Error calculating isochrones: {e}")
        return gpd.GeoDataFrame()

# Calculate isochrones for each fire station (in minutes)
response_times = [10, 20, 30]
all_isochrones = []

print("Calculating response time isochrones...")
for idx, station in fire_stations.iterrows():
    print(f"  Processing {station['station_name']}...")
    isochrones = calculate_isochrones(
        G, 
        station.geometry, 
        response_times
    )
    
    if not isochrones.empty:
        isochrones['station_name'] = station['station_name']
        isochrones['agency'] = station['agency']
        all_isochrones.append(isochrones)

if all_isochrones:
    isochrones_gdf = pd.concat(all_isochrones, ignore_index=True)
    print(f"\n✓ Calculated {len(isochrones_gdf)} isochrones")
else:
    print("\nNo isochrones calculated")
    isochrones_gdf = gpd.GeoDataFrame()

## Coverage Gap Analysis

### Identify Uncovered Tribal Lands

In [ ]:
def analyze_coverage_gaps(tribal_lands, isochrones, max_response_time=30):
    """
    Analyze which Tribal lands fall outside reasonable response times.
    
    Parameters:
    tribal_lands: GeoDataFrame of Tribal areas
    isochrones: GeoDataFrame of service area polygons
    max_response_time: maximum acceptable response time in minutes
    
    Returns:
    Dictionary with coverage statistics and GeoDataFrames
    """
    # Filter isochrones to max response time
    coverage_areas = isochrones[isochrones['time_minutes'] <= max_response_time]
    
    if coverage_areas.empty:
        print("No coverage areas available")
        return None
    
    # Create union of all coverage areas
    total_coverage = coverage_areas.dissolve().geometry.iloc[0]
    
    # Calculate coverage for each tribal area
    results = []
    
    for idx, tribe in tribal_lands.iterrows():
        # Calculate intersection
        covered = tribe.geometry.intersection(total_coverage)
        covered_area = covered.area if not covered.is_empty else 0
        total_area = tribe.geometry.area
        
        coverage_pct = (covered_area / total_area * 100) if total_area > 0 else 0
        
        # Calculate gap area
        gap = tribe.geometry.difference(total_coverage)
        gap_area = gap.area if not gap.is_empty else 0
        
        results.append({
            'tribe_name': tribe['tribe_name'],
            'has_fire_program': tribe['has_fire_program'],
            'fire_staff_count': tribe['fire_staff_count'],
            'total_area_sq_deg': total_area,
            'covered_area_sq_deg': covered_area,
            'gap_area_sq_deg': gap_area,
            'coverage_percent': coverage_pct,
            'geometry': tribe.geometry,
            'gap_geometry': gap if not gap.is_empty else None
        })
    
    coverage_analysis = gpd.GeoDataFrame(results, crs=tribal_lands.crs)
    
    # Summary statistics
    summary = {
        'total_tribal_areas': len(tribal_lands),
        'fully_covered': len(coverage_analysis[coverage_analysis['coverage_percent'] >= 99]),
        'partially_covered': len(coverage_analysis[
            (coverage_analysis['coverage_percent'] > 0) & 
            (coverage_analysis['coverage_percent'] < 99)
        ]),
        'no_coverage': len(coverage_analysis[coverage_analysis['coverage_percent'] == 0]),
        'avg_coverage_pct': coverage_analysis['coverage_percent'].mean(),
        'areas_with_own_programs': len(coverage_analysis[coverage_analysis['has_fire_program'] == True]),
        'areas_without_programs': len(coverage_analysis[coverage_analysis['has_fire_program'] == False])
    }
    
    return {
        'coverage_analysis': coverage_analysis,
        'summary': summary,
        'total_coverage_polygon': total_coverage
    }

# Run coverage analysis
if not isochrones_gdf.empty:
    gap_analysis = analyze_coverage_gaps(tribal_lands, isochrones_gdf, max_response_time=30)
    
    if gap_analysis:
        print("\n" + "="*60)
        print("COVERAGE GAP ANALYSIS SUMMARY")
        print("="*60)
        for key, value in gap_analysis['summary'].items():
            print(f"{key.replace('_', ' ').title()}: {value:.2f}" if isinstance(value, float) else f"{key.replace('_', ' ').title()}: {value}")
        print("="*60)
        
        print("\nDetailed Coverage by Tribal Area:")
        display(gap_analysis['coverage_analysis'][[
            'tribe_name', 'has_fire_program', 'fire_staff_count', 
            'coverage_percent'
        ]].sort_values('coverage_percent'))
else:
    print("Cannot perform gap analysis without isochrone data")
    gap_analysis = None

### Prioritize Areas for Improvement

In [ ]:
def prioritize_gaps(coverage_analysis):
    """
    Create priority ranking for areas needing fire response improvements.
    
    Priority factors:
    - Coverage gap size
    - Lack of tribal fire program
    - Low staffing levels
    """
    df = coverage_analysis.copy()
    
    # Calculate priority score (higher = more urgent)
    df['priority_score'] = 0.0
    
    # Coverage gap (0-40 points)
    df['priority_score'] += (100 - df['coverage_percent']) * 0.4
    
    # No tribal fire program (30 points)
    df['priority_score'] += (~df['has_fire_program']) * 30
    
    # Low staffing (0-30 points)
    max_staff = df['fire_staff_count'].max()
    if max_staff > 0:
        df['priority_score'] += (1 - df['fire_staff_count'] / max_staff) * 30
    
    # Assign priority categories
    df['priority_category'] = pd.cut(
        df['priority_score'],
        bins=[0, 30, 60, 100],
        labels=['Low', 'Medium', 'High']
    )
    
    return df.sort_values('priority_score', ascending=False)

if gap_analysis:
    priority_analysis = prioritize_gaps(gap_analysis['coverage_analysis'])
    
    print("\nPRIORITY RANKING FOR FIRE RESPONSE IMPROVEMENTS")
    print("="*80)
    display(priority_analysis[[
        'tribe_name', 'coverage_percent', 'has_fire_program', 
        'fire_staff_count', 'priority_score', 'priority_category'
    ]])
    
    print(f"\nHigh Priority Areas: {len(priority_analysis[priority_analysis['priority_category'] == 'High'])}")
    print(f"Medium Priority Areas: {len(priority_analysis[priority_analysis['priority_category'] == 'Medium'])}")
    print(f"Low Priority Areas: {len(priority_analysis[priority_analysis['priority_category'] == 'Low'])}")

## Visualization

### Static Map: Coverage Overview

In [ ]:
# Define output directory
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Use modern matplotlib style
try:
    plt.style.use('seaborn-v0_8-darkgrid') 
except:
    plt.style.use('ggplot')

# Create dashboard
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Ownership Types
sorted_data = complexity_analysis.sort_values('num_ownership_types', ascending=True)
axes[0, 0].barh(sorted_data['reservation_name'], sorted_data['num_ownership_types'],
                color='steelblue', alpha=0.7)
axes[0, 0].set_xlabel('Number of Ownership Types', fontweight='bold')
axes[0, 0].set_title('Land Ownership Diversity', fontweight='bold', fontsize=14)
axes[0, 0].grid(True, alpha=0.3, axis='x')

# Number of Fire Authorities
sorted_data = complexity_analysis.sort_values('num_authorities', ascending=True)
axes[0, 1].barh(sorted_data['reservation_name'], sorted_data['num_authorities'],
                color='coral', alpha=0.7)
axes[0, 1].set_xlabel('Number of Fire Authorities', fontweight='bold')
axes[0, 1].set_title('Fire Authority Overlap', fontweight='bold', fontsize=14)
axes[0, 1].grid(True, alpha=0.3, axis='x')

# Mutual Aid Coverage
sorted_data = complexity_analysis.sort_values('mutual_aid_pct', ascending=True)
colors = ['red' if x < 50 else 'yellow' if x < 80 else 'green' 
          for x in sorted_data['mutual_aid_pct']]
axes[1, 0].barh(sorted_data['reservation_name'], sorted_data['mutual_aid_pct'],
                color=colors, alpha=0.7)
axes[1, 0].set_xlabel('Mutual Aid Coverage (%)', fontweight='bold')
axes[1, 0].set_title('Mutual Aid Agreement Coverage', fontweight='bold', fontsize=14)
axes[1, 0].axvline(80, color='green', linestyle='--', label='80% Target')
axes[1, 0].grid(True, alpha=0.3, axis='x')
axes[1, 0].legend()

# Fragmentation Index
sorted_data = complexity_analysis.sort_values('fragmentation_index', ascending=True)
colors = ['green' if x < 40 else 'yellow' if x < 65 else 'red' 
          for x in sorted_data['fragmentation_index']]
axes[1, 1].barh(sorted_data['reservation_name'], sorted_data['fragmentation_index'],
                color=colors, alpha=0.7)
axes[1, 1].set_xlabel('Fragmentation Index', fontweight='bold')
axes[1, 1].set_title('Overall Land Fragmentation', fontweight='bold', fontsize=14)
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.suptitle('Jurisdictional Fragmentation Metrics Dashboard',
             fontsize=18, fontweight='bold', y=1.00)

plt.tight_layout()

# Save figure
save_path = os.path.join(output_dir, "fragmentation_dashboard.png")
plt.savefig(save_path, dpi=300, bbox_inches='tight')

plt.show()

print(f"\nDashboard saved to: {save_path}")

# Reset style to default for future plots
plt.style.use('default')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 12))

# Plot tribal lands
tribal_lands.plot(
    ax=ax, 
    color='lightblue', 
    edgecolor='blue', 
    alpha=0.3,
    label='Tribal Lands'
)

# Plot isochrones with different colors for different time ranges
if not isochrones_gdf.empty:
    colors = {10: 'green', 20: 'yellow', 30: 'orange'}
    for time in sorted(isochrones_gdf['time_minutes'].unique()):
        isochrones_gdf[isochrones_gdf['time_minutes'] == time].plot(
            ax=ax,
            color=colors.get(time, 'gray'),
            alpha=0.2,
            edgecolor='black',
            linewidth=0.5,
            label=f'{time} min response'
        )

# Plot fire stations
fire_stations.plot(
    ax=ax,
    color='red',
    marker='*',
    markersize=200,
    edgecolor='black',
    label='Fire Stations',
    zorder=5
)

# Add labels for fire stations
for idx, station in fire_stations.iterrows():
    ax.annotate(
        station['station_name'],
        xy=(station.geometry.x, station.geometry.y),
        xytext=(5, 5),
        textcoords='offset points',
        fontsize=8,
        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7)
    )

# Add basemap
try:
    ctx.add_basemap(
        ax, 
        crs=tribal_lands.crs.to_string(),
        source=ctx.providers.OpenStreetMap.Mapnik,
        alpha=0.5
    )
except:
    pass

ax.set_title('Fire Response Coverage for Tribal Lands', fontsize=16, fontweight='bold')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/home/claude/fire_coverage_map.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nMap saved to: fire_coverage_map.png")

### Priority Areas Visualization

In [ ]:
if gap_analysis:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Chart 1: Coverage by Tribal Area
    coverage_data = gap_analysis['coverage_analysis'].sort_values('coverage_percent')
    
    colors_chart = ['red' if not x else 'green' 
                    for x in coverage_data['has_fire_program']]
    
    ax1.barh(coverage_data['tribe_name'], coverage_data['coverage_percent'], 
             color=colors_chart, alpha=0.7)
    ax1.axvline(x=90, color='orange', linestyle='--', label='90% Coverage Target')
    ax1.set_xlabel('Coverage Percentage (%)', fontsize=12)
    ax1.set_title('Fire Response Coverage by Tribal Area', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='x')
    
    # Chart 2: Fire Program Status
    program_counts = gap_analysis['coverage_analysis']['has_fire_program'].value_counts()
    
    ax2.pie(program_counts, labels=['Has Fire Program', 'No Fire Program'],
            autopct='%1.1f%%', colors=['green', 'red'], startangle=90)
    ax2.set_title('Tribal Fire Program Status', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('/home/claude/coverage_analysis_charts.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\nCharts saved to: coverage_analysis_charts.png")

## Export Results

### Export Spatial Data

In [ ]:
# Export tribal lands with coverage stats
if gap_analysis:
    coverage_export = gap_analysis['coverage_analysis'][[
        'tribe_name', 'has_fire_program', 'fire_staff_count',
        'coverage_percent', 'geometry'
    ]]
    coverage_export.to_file(
        f'{output_dir}/tribal_coverage_analysis.geojson',
        driver='GeoJSON'
    )
    print(f"Exported: {output_dir}/tribal_coverage_analysis.geojson")

# Export fire stations
fire_stations.to_file(
    f'{output_dir}/fire_stations.geojson',
    driver='GeoJSON'
)
print(f"Exported: {output_dir}/fire_stations.geojson")

# Export isochrones
if not isochrones_gdf.empty:
    isochrones_gdf.to_file(
        f'{output_dir}/response_time_isochrones.geojson',
        driver='GeoJSON'
    )
    print(f"Exported: {output_dir}/response_time_isochrones.geojson")

# Export priority analysis
if gap_analysis:
    priority_export = prioritize_gaps(gap_analysis['coverage_analysis'])[[
        'tribe_name', 'coverage_percent', 'has_fire_program',
        'fire_staff_count', 'priority_score', 'priority_category', 'geometry'
    ]]
    priority_export.to_file(
        f'{output_dir}/priority_areas.geojson',
        driver='GeoJSON'
    )
    print(f"Exported: {output_dir}/priority_areas.geojson")

### Export Summary Report

In [ ]:
# Create summary report
if gap_analysis:
    report = f"""
FIRE RESPONSE & MANAGEMENT CAPACITY GAP ANALYSIS
Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}
{'='*80}

EXECUTIVE SUMMARY
{'='*80}

Tribal Areas Analyzed: {gap_analysis['summary']['total_tribal_areas']}
Fire Stations Mapped: {len(fire_stations)}
  - BIA Stations: {len(fire_stations[fire_stations['agency'] == 'BIA'])}
  - USFS Stations: {len(fire_stations[fire_stations['agency'] == 'USFS'])}
  - BLM Stations: {len(fire_stations[fire_stations['agency'] == 'BLM'])}
  - State Stations: {len(fire_stations[fire_stations['agency'] == 'State'])}

COVERAGE ANALYSIS (30-minute response time)
{'='*80}

Fully Covered (≥99%): {gap_analysis['summary']['fully_covered']} areas
Partially Covered (1-98%): {gap_analysis['summary']['partially_covered']} areas
No Coverage (0%): {gap_analysis['summary']['no_coverage']} areas

Average Coverage: {gap_analysis['summary']['avg_coverage_pct']:.1f}%

TRIBAL FIRE PROGRAMS
{'='*80}

Areas with Fire Programs: {gap_analysis['summary']['areas_with_own_programs']}
Areas without Programs: {gap_analysis['summary']['areas_without_programs']}

DETAILED COVERAGE BY AREA
{'='*80}
"""
    
    for idx, row in gap_analysis['coverage_analysis'].sort_values('coverage_percent').iterrows():
        report += f"""
{row['tribe_name']}:
  Coverage: {row['coverage_percent']:.1f}%
  Fire Program: {'Yes' if row['has_fire_program'] else 'No'}
  Staff Count: {row['fire_staff_count']}
"""
    
    # Save report
    report_path = f'{output_dir}/fire_capacity_gap_report.txt'
    with open(report_path, 'w') as f:
        f.write(report)
    
    print(f"\n✓ Summary report saved to: {report_path}")
    print("\n" + report)

### Generate Recommendations

In [ ]:
def generate_recommendations(gap_analysis, priority_analysis):
    """
    Generate actionable recommendations based on analysis.
    """
    recommendations = []
    
    # High priority areas without fire programs
    high_priority_no_program = priority_analysis[
        (priority_analysis['priority_category'] == 'High') & 
        (~priority_analysis['has_fire_program'])
    ]
    
    if len(high_priority_no_program) > 0:
        recommendations.append({
            'priority': 'CRITICAL',
            'category': 'Tribal Fire Program Development',
            'description': f"Establish fire programs for {len(high_priority_no_program)} high-priority tribal areas currently without programs",
            'affected_areas': high_priority_no_program['tribe_name'].tolist()
        })
    
    # Areas with poor coverage
    poor_coverage = gap_analysis['coverage_analysis'][
        gap_analysis['coverage_analysis']['coverage_percent'] < 50
    ]
    
    if len(poor_coverage) > 0:
        recommendations.append({
            'priority': 'HIGH',
            'category': 'Station Placement',
            'description': f"Consider additional fire stations or mutual aid agreements for {len(poor_coverage)} areas with <50% coverage",
            'affected_areas': poor_coverage['tribe_name'].tolist()
        })
    
    # Areas with programs but low staffing
    understaffed = gap_analysis['coverage_analysis'][
        (gap_analysis['coverage_analysis']['has_fire_program']) &
        (gap_analysis['coverage_analysis']['fire_staff_count'] < 10)
    ]
    
    if len(understaffed) > 0:
        recommendations.append({
            'priority': 'MEDIUM',
            'category': 'Staffing Enhancement',
            'description': f"Increase staffing levels for {len(understaffed)} tribal fire programs with <10 personnel",
            'affected_areas': understaffed['tribe_name'].tolist()
        })
    
    # Road network improvements
    slow_access = gap_analysis['coverage_analysis'][
        gap_analysis['coverage_analysis']['coverage_percent'] < 75
    ]
    
    if len(slow_access) > 0:
        recommendations.append({
            'priority': 'MEDIUM',
            'category': 'Infrastructure',
            'description': f"Assess road network improvements to enhance response times for {len(slow_access)} areas",
            'affected_areas': slow_access['tribe_name'].tolist()
        })
    
    return recommendations

if gap_analysis and 'coverage_analysis' in gap_analysis:
    recommendations = generate_recommendations(gap_analysis, priority_analysis)
    
    print("\n" + "="*80)
    print("RECOMMENDATIONS FOR FIRE RESPONSE CAPACITY IMPROVEMENTS")
    print("="*80)
    
    for i, rec in enumerate(recommendations, 1):
        print(f"\n{i}. [{rec['priority']}] {rec['category']}")
        print(f"   {rec['description']}")
        print(f"   Affected Areas: {', '.join(rec['affected_areas'])}")
    
    # Save recommendations
    rec_df = pd.DataFrame(recommendations)
    rec_df.to_csv(f'{output_dir}/recommendations.csv', index=False)
    print(f"\nRecommendations saved to: {output_dir}/recommendations.csv")

## Next Steps and Data Sources

### Data Sources to Integrate:

**Tribal Lands Boundaries**
   - BIA AIAN Lands Dataset: https://biamaps.doi.gov/
   - Census TIGER/Line American Indian Areas

**Fire Station Locations**
   - BIA Division of Fire Management
   - USFS Fire & Aviation Management
   - State forestry departments
   - HIFLD Fire Stations dataset

**Tribal Fire Programs**
   - Contact individual tribal governments
   - BIA tribal fire program directories
   - Inter-Tribal Timber Council resources

**Road Networks**
   - OpenStreetMap (used in this notebook)
   - TIGER/Line Roads
   - USFS Road Network

### Recommended Enhancements:

- Integrate wildfire risk data (LANDFIRE, Wildfire Hazard Potential)
- Add population density for risk prioritization
- Include seasonal staffing variations
- Model different emergency scenarios (structure fire vs wildland)
- Analyze water source accessibility
- Consider helicopter response capabilities
- Add mutual aid agreement mapping

### Usage Notes:

This notebook provides a framework that can be customized with real data. The sample data demonstrates the methodology but should be replaced with:
- Actual Tribal boundary shapefiles
- Verified fire station locations and capabilities
- Current Tribal fire program information
- Local road network conditions